This script trains an LSTM model on one of the data files in the data folder of
this repository. the input file can be changed to another file from the data folder
by changing its name in line 46.

it is recommended to run this script on GPU, as recurrent networks are quite 
computationally intensive.

Author: Niek Tax

In [1]:
from keras.models import Sequential, Model
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers import Input
from keras.utils.data_utils import get_file
from keras.regularizers import WeightRegularizer
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization

Using Theano backend.


In [2]:
from __future__ import print_function, division
from collections import Counter
import unicodecsv
import numpy as np
import random
import sys
import os
import copy
import csv
import time
from itertools import izip
from datetime import datetime
from math import log

# Part 1

Steps to create features:
1. timeseqs: duration of each activity in case - [[0, 15, 173999], [0, 457325, 306], [0, 62954, 432], [0, 2221, 109, 1660, 6746], [0, 2697, 64551], [0, 3313, 1141], [0, 13, 1459434, 104956], [0, 3196606]]
--> divisor: mean
2. timeseqs2: cumulative duration of activities in case - [[0, 15, 174014], [0, 457325, 457631], [0, 62954, 63386], [0, 2221, 2330, 3990, 10736], [0, 2697, 67248], [0, 3313, 4454], [0, 13, 1459447, 1564403], [0, 3196606]]
--> divisor2: mean
3. numlines: total number of cases - 9
4. line: activities - [u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa4\xa2\xa9\xa7', u'\xa2\xa9']

Create 3 folds:
1. FOLD 1
    - fold1 <-- lines
    - fold1_t <-- timeseqs
    - fold1_t2 <-- timeseqs2

2. FOLD 2
    - fold2 <-- lines
    - fold2_t <-- timeseqs
    - fold2_t2 <-- timeseqs2

3. FOLD 3
    - fold2 <-- lines
    - fold2_t <-- timeseqs
    - fold2_t2 <-- timeseqs2

**NOTE** Fold1 and fold2 were combined to use for training, fold3 were used for testing
1. train:
    - lines = fold1 + fold2
    - lines_t = fold1_t + fold2_t
    - lines_t2 = fold1_t2 + fold2_t2
2. Create dict for char_to_indice, indice_to_char

In [3]:
lastcase = ''
line = ''
firstLine = True
lines = []
timeseqs = []
timeseqs2 = []
times = []
times2 = []
numlines = 0
casestarttime = None
lasteventtime = None
eventlog = "helpdesk.csv"

In [4]:
csvfile = open('../data/%s' % eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers
ascii_offset = 161

```python
#Run this function to see how time package works
t1 = time.strptime('2012-04-03 16:55:38', "%Y-%m-%d %H:%M:%S")
t2 = time.strptime('2012-04-03 16:55:53', "%Y-%m-%d %H:%M:%S")
t3 = time.mktime(t1)  # This method returns a floating point number, for compatibility with time()
t4 = time.mktime(t2)
t5 = datetime.fromtimestamp(t4) - datetime.fromtimestamp(t3)
t6 = t5.days
t7 = t5.seconds
print('t1: {}'.format(t1))
print('t2: {}'.format(t2))
print('t3: {}'.format(t3))
print('t5: {}'.format(t5))
print('t6: {}'.format(t6))
print('t7: {}'.format(t7))
```

In [5]:
i = 0
for row in spamreader:
    print ('\n')
    print("i: {}".format(i))
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S")
    print('t: {}'.format(t))
    #This if check whether the activity is still in current case --> Enter if only move to the next case
    if row[0]!=lastcase: 
        print('ENTER IF 1')
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        print('casestarttime: {}'.format(casestarttime))
        print('lasteventtime: {}'.format(lasteventtime))
        print('lastcase: {}'.format(lastcase))
        
        if not firstLine:
            print('ENTER IF 2')
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
            
            #print('lines: {}'.format(lines))
            print('timeseqs: {}'.format(timeseqs))
            print('timeseqs2: {}'.format(timeseqs2))
            
        line = ''
        times = []
        times2 = []
        numlines+=1
        
        print('line: {}'.format(line))
        print('times: {}'.format(times))
        print('times2: {}'.format(times2))
        print('numlines: {}'.format(numlines))
        
    line+=unichr(int(row[1])+ascii_offset)
    
    
    #print('line: {}'.format(line))
    
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds
    times.append(timediff)
    times2.append(timediff2)
    lasteventtime = t
    
    print('timesincelastevent: {}'.format(timesincelastevent))
    print('timesincecasestart: {}'.format(timesincecasestart))
    print('timediff: {}'.format(timediff))
    print('timediff2: {}'.format(timediff2))
    print('times: {}'.format(times))
    print('times2: {}'.format(times2))
    print('lasteventtime: {}'.format(lasteventtime))
    
    firstLine = False
    i+=1
    if i == 50:
        break



i: 0
t: time.struct_time(tm_year=2012, tm_mon=4, tm_mday=3, tm_hour=16, tm_min=55, tm_sec=38, tm_wday=1, tm_yday=94, tm_isdst=-1)
ENTER IF 1
casestarttime: time.struct_time(tm_year=2012, tm_mon=4, tm_mday=3, tm_hour=16, tm_min=55, tm_sec=38, tm_wday=1, tm_yday=94, tm_isdst=-1)
lasteventtime: time.struct_time(tm_year=2012, tm_mon=4, tm_mday=3, tm_hour=16, tm_min=55, tm_sec=38, tm_wday=1, tm_yday=94, tm_isdst=-1)
lastcase: 2
line: 
times: []
times2: []
numlines: 1
timesincelastevent: 0:00:00
timesincecasestart: 0:00:00
timediff: 0
timediff2: 0
times: [0]
times2: [0]
lasteventtime: time.struct_time(tm_year=2012, tm_mon=4, tm_mday=3, tm_hour=16, tm_min=55, tm_sec=38, tm_wday=1, tm_yday=94, tm_isdst=-1)


i: 1
t: time.struct_time(tm_year=2012, tm_mon=4, tm_mday=3, tm_hour=16, tm_min=55, tm_sec=53, tm_wday=1, tm_yday=94, tm_isdst=-1)
timesincelastevent: 0:00:15
timesincecasestart: 0:00:15
timediff: 15
timediff2: 15
times: [0, 15]
times2: [0, 15]
lasteventtime: time.struct_time(tm_year=2012

In [6]:
# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
numlines+=1

In [7]:
numlines

16

In [8]:
divisor = np.mean([item for sublist in timeseqs for item in sublist])
print('divisor: {}'.format(divisor))
divisor2 = np.mean([item for sublist in timeseqs2 for item in sublist])
print('divisor2: {}'.format(divisor2))

divisor: 245344.38
divisor2: 408663.02


In [9]:
for sublist in timeseqs:
    for item in sublist:
        print(item)

0
15
173999
0
457325
306
0
62954
432
0
2221
109
1660
6746
0
2697
64551
0
3313
1141
0
13
1459434
104956
0
3196606
7
0
53
66609
0
14
315
0
7035
590
62621
0
1678025
594825
0
181008
26
0
1097476
7958
0
10
3765
3028404


In [10]:
print('numlines: {}'.format(numlines))

numlines: 16


In [11]:
print('lines: {}'.format(lines))
print('timeseqs: {}'.format(timeseqs))
print('timeseqs2: {}'.format(timeseqs2))

lines: [u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa4\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xaa\xa7']
timeseqs: [[0, 15, 173999], [0, 457325, 306], [0, 62954, 432], [0, 2221, 109, 1660, 6746], [0, 2697, 64551], [0, 3313, 1141], [0, 13, 1459434, 104956], [0, 3196606, 7], [0, 53, 66609], [0, 14, 315], [0, 7035, 590, 62621], [0, 1678025, 594825], [0, 181008, 26], [0, 1097476, 7958], [0, 10, 3765, 3028404]]
timeseqs2: [[0, 15, 174014], [0, 457325, 457631], [0, 62954, 63386], [0, 2221, 2330, 3990, 10736], [0, 2697, 67248], [0, 3313, 4454], [0, 13, 1459447, 1564403], [0, 3196606, 3196613], [0, 53, 66662], [0, 14, 329], [0, 7035, 7625, 70246], [0, 1678025, 2272850], [0, 181008, 181034], [0, 1097476, 1105434], [0, 10, 3775, 3032179]]


In [12]:
elems_per_fold = int(round(numlines/3))
fold1 = lines[:elems_per_fold]
fold1_t = timeseqs[:elems_per_fold]
fold1_t2 = timeseqs2[:elems_per_fold]

fold2 = lines[elems_per_fold:2*elems_per_fold]
fold2_t = timeseqs[elems_per_fold:2*elems_per_fold]
fold2_t2 = timeseqs2[elems_per_fold:2*elems_per_fold]

fold3 = lines[2*elems_per_fold:]
fold3_t = timeseqs[2*elems_per_fold:]
fold3_t2 = timeseqs2[2*elems_per_fold:]

In [13]:
print('fold1: {}'.format(fold1))
print('fold1_t: {}'.format(fold1_t))
print('fold1_t2: {}'.format(fold1_t2))

fold1: [u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7\xa9\xa7', u'\xa2\xa9\xa7']
fold1_t: [[0, 15, 173999], [0, 457325, 306], [0, 62954, 432], [0, 2221, 109, 1660, 6746], [0, 2697, 64551]]
fold1_t2: [[0, 15, 174014], [0, 457325, 457631], [0, 62954, 63386], [0, 2221, 2330, 3990, 10736], [0, 2697, 67248]]


In [14]:
lines = fold1 + fold2
lines_t = fold1_t + fold2_t
lines_t2 = fold1_t2 + fold2_t2

In [15]:
print('lines: {}'.format(lines))
print('lines_t: {}'.format(lines_t))
print('lines_t2: {}'.format(lines_t2))

lines: [u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa4\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7']
lines_t: [[0, 15, 173999], [0, 457325, 306], [0, 62954, 432], [0, 2221, 109, 1660, 6746], [0, 2697, 64551], [0, 3313, 1141], [0, 13, 1459434, 104956], [0, 3196606, 7], [0, 53, 66609], [0, 14, 315]]
lines_t2: [[0, 15, 174014], [0, 457325, 457631], [0, 62954, 63386], [0, 2221, 2330, 3990, 10736], [0, 2697, 67248], [0, 3313, 4454], [0, 13, 1459447, 1564403], [0, 3196606, 3196613], [0, 53, 66662], [0, 14, 329]]


In [16]:
step = 1
sentences = []
softness = 0
next_chars = []
lines = map(lambda x: x+'!',lines)
maxlen = max(map(lambda x: len(x),lines)) #return len of longest case

In [17]:
print(lines)

[u'\xa2\xa9\xa7!', u'\xa2\xa9\xa7!', u'\xa2\xa9\xa7!', u'\xa2\xa9\xa7\xa9\xa7!', u'\xa2\xa9\xa7!', u'\xa2\xa9\xa7!', u'\xa4\xa2\xa9\xa7!', u'\xa2\xa9\xa7!', u'\xa2\xa9\xa7!', u'\xa2\xa9\xa7!']


In [18]:
chars = map(lambda x : set(x),lines)
chars = list(set().union(*chars))
chars.sort()
target_chars = copy.copy(chars)
chars.remove('!')
print('total chars: {}, target chars: {}'.format(len(chars), len(target_chars)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i, c) for i, c in enumerate(target_chars))
print('indices_char: {}'.format(indices_char))
print('chars: {}'.format(chars))

total chars: 4, target chars: 5
indices_char: {0: u'\xa2', 1: u'\xa4', 2: u'\xa7', 3: u'\xa9'}
chars: [u'\xa2', u'\xa4', u'\xa7', u'\xa9']


In [19]:
print('lines: {}'.format(lines))

lines: [u'\xa2\xa9\xa7!', u'\xa2\xa9\xa7!', u'\xa2\xa9\xa7!', u'\xa2\xa9\xa7\xa9\xa7!', u'\xa2\xa9\xa7!', u'\xa2\xa9\xa7!', u'\xa4\xa2\xa9\xa7!', u'\xa2\xa9\xa7!', u'\xa2\xa9\xa7!', u'\xa2\xa9\xa7!']


In [20]:
print('maxlen: {}'.format(maxlen))

maxlen: 6


In [21]:
print('char_indices: {}'.format(char_indices))
print('indices_char: {}'.format(indices_char))
print('target_char_indices: {}'.format(target_char_indices))
print('target_indices_char: {}'.format(target_indices_char))

char_indices: {u'\xa9': 3, u'\xa2': 0, u'\xa4': 1, u'\xa7': 2}
indices_char: {0: u'\xa2', 1: u'\xa4', 2: u'\xa7', 3: u'\xa9'}
target_char_indices: {u'!': 0, u'\xa9': 4, u'\xa2': 1, u'\xa4': 2, u'\xa7': 3}
target_indices_char: {0: u'!', 1: u'\xa2', 2: u'\xa4', 3: u'\xa7', 4: u'\xa9'}


In [22]:
#unichr(6)+unichr(8)+'!'

# Part 2

- timediff: Time difference since last activity --> timeseqs
- timediff2: Time difference since starting time of a case --> timeseqs2
- timediff3: Time difference since midnight --> timeseqs3
- timediff4: weekday --> timeseqs4
- lines: [ABC, ABC, ABC]
- numline:number of cases

Steps to create features:
1. timeseqs: duration of each activity in case - [[0, 15, 173999], [0, 457325, 306], [0, 62954, 432], [0, 2221, 109, 1660, 6746], [0, 2697, 64551], [0, 3313, 1141], [0, 13, 1459434, 104956], [0, 3196606]]
--> divisor: mean
2. timeseqs2: cumulative duration of activities in case - [[0, 15, 174014], [0, 457325, 457631], [0, 62954, 63386], [0, 2221, 2330, 3990, 10736], [0, 2697, 67248], [0, 3313, 4454], [0, 13, 1459447, 1564403], [0, 3196606]]
--> divisor2: mean
3. timeseq3: time different from midnight - [[60938, 60953, 62152], [65646, 4571, 4877], [84713, 61267, 61699], [76112, 78333, 78442, 80102, 448], [83638, 86335, 64486], [59432, 62745, 63886], [75741, 75754, 66388, 84944], [64220, 64026]]
4. timeseq4: weekday - [[1, 1, 3], [4, 3, 3], [2, 3, 3], [1, 1, 1, 1, 2], [1, 1, 2], [3, 3, 3], [4, 4, 0, 1], [2, 4]]
5. numlines: total number of cases - 9
6. line: activities - [u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa4\xa2\xa9\xa7', u'\xa2\xa9']

Create 3 folds:
1. FOLD 1
    - fold1 <-- lines
    - fold1_t <-- timeseqs
    - fold1_t2 <-- timeseqs2
    - fold1_t3 <-- timeseqs3
    - fold1_t4 <-- timeseqs4

2. FOLD 2
    - fold2 <-- lines
    - fold2_t <-- timeseqs
    - fold2_t2 <-- timeseqs2
    - fold2_t3 <-- timeseqs3
    - fold2_t4 <-- timeseqs4

3. FOLD 3
    - fold2 <-- lines
    - fold2_t <-- timeseqs
    - fold2_t2 <-- timeseqs2
    - fold2_t3 <-- timeseqs3
    - fold2_t4 <-- timeseqs4

**NOTE** Fold1 and fold2 were combined to use for training, fold3 were used for testing
1. train:
    - lines = fold1 + fold2 #activity
    - lines_t = fold1_t + fold2_t #time
    - lines_t2 = fold1_t2 + fold2_t2
    - lines_t3 = fold1_t3 + fold2_t3
    - lines_t4 = fold1_t4 + fold2_t4 #weekday

- Input: sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4
- Output: next_chars, next_chars_t, next_chars_t2, next_chars_t3, next_chars_t4

In [23]:
lastcase = ''
line = ''
firstLine = True
lines = []
timeseqs = []
timeseqs2 = []
timeseqs3 = []
timeseqs4 = []
times = []
times2 = []
times3 = []
times4 = []
numlines = 0
casestarttime = None
lasteventtime = None
eventlog = "helpdesk.csv"

In [24]:
csvfile = open('../data/%s' % eventlog, 'r')
spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
next(spamreader, None)  # skip the headers
ascii_offset = 161

In [25]:
i = 0
for row in spamreader:
    print ('\n')
    print("i: {}".format(i))
    t = time.strptime(row[2], "%Y-%m-%d %H:%M:%S")
    if row[0]!=lastcase:
        casestarttime = t
        lasteventtime = t
        lastcase = row[0]
        
        print('--------------Check----------------')
        print('casestarttime: {}'.format(casestarttime))
        print('lasteventtime: {}'.format(lasteventtime))
        print('lastcase: {}'.format(lastcase))
        
        if not firstLine: 
            
            print('--------------Check----------------')
            print("Enter IF:")
            lines.append(line)
            timeseqs.append(times)
            timeseqs2.append(times2)
            timeseqs3.append(times3)
            timeseqs4.append(times4)
            
            print('lines: {}'.format(lines))
            print('timeseqs: {}'.format(timeseqs))
            print('timeseqs2: {}'.format(timeseqs2))
            print('timeseqs3: {}'.format(timeseqs3))
            print('timeseqs4: {}'.format(timeseqs4))
            
        line = ''
        times = []
        times2 = []
        times3 = []
        times4 = []
        numlines+=1
        
    line+=unichr(int(row[1])+ascii_offset)
    #print('line: {}'.format(line))
    
    timesincelastevent = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(lasteventtime))
    timesincecasestart = datetime.fromtimestamp(time.mktime(t))-datetime.fromtimestamp(time.mktime(casestarttime))
    
    midnight = datetime.fromtimestamp(time.mktime(t)).replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = datetime.fromtimestamp(time.mktime(t))-midnight
    
    timediff = 86400 * timesincelastevent.days + timesincelastevent.seconds
    timediff2 = 86400 * timesincecasestart.days + timesincecasestart.seconds
    timediff3 = timesincemidnight.seconds
    timediff4 = datetime.fromtimestamp(time.mktime(t)).weekday()
    
    print('--------------Check----------------')
    print('timediff: {}'.format(timediff))
    print('timediff2: {}'.format(timediff2))
    print('timediff3: {}'.format(timediff3))
    print('timediff4: {}'.format(timediff4))
    
    
    times.append(timediff)
    times2.append(timediff2)
    times3.append(timediff3)
    times4.append(timediff4)
    
    print('--------------Check----------------')
    print('times: {}'.format(times))
    print('times2: {}'.format(times2))
    print('times3: {}'.format(times3))
    print('times4: {}'.format(times4))
    
    lasteventtime = t
    firstLine = False
    i+=1
    if i == 50:
        break



i: 0
--------------Check----------------
casestarttime: time.struct_time(tm_year=2012, tm_mon=4, tm_mday=3, tm_hour=16, tm_min=55, tm_sec=38, tm_wday=1, tm_yday=94, tm_isdst=-1)
lasteventtime: time.struct_time(tm_year=2012, tm_mon=4, tm_mday=3, tm_hour=16, tm_min=55, tm_sec=38, tm_wday=1, tm_yday=94, tm_isdst=-1)
lastcase: 2
--------------Check----------------
timediff: 0
timediff2: 0
timediff3: 60938
timediff4: 1
--------------Check----------------
times: [0]
times2: [0]
times3: [60938]
times4: [1]


i: 1
--------------Check----------------
timediff: 15
timediff2: 15
timediff3: 60953
timediff4: 1
--------------Check----------------
times: [0, 15]
times2: [0, 15]
times3: [60938, 60953]
times4: [1, 1]


i: 2
--------------Check----------------
timediff: 173999
timediff2: 174014
timediff3: 62152
timediff4: 3
--------------Check----------------
times: [0, 15, 173999]
times2: [0, 15, 174014]
times3: [60938, 60953, 62152]
times4: [1, 1, 3]


i: 3
--------------Check----------------
casest

```python
#run this code to see weekday()
import datetime
dt = '03/04/2012'
day, month, year = (int(x) for x in dt.split('/'))    
ans = datetime.date(int(year),int(month),int(day))
ans.weekday()
```

In [26]:
# add last case
lines.append(line)
timeseqs.append(times)
timeseqs2.append(times2)
timeseqs3.append(times3)
timeseqs4.append(times4)
numlines+=1

In [27]:
elems_per_fold = int(round(numlines/3))
fold1 = lines[:elems_per_fold] #activity
fold1_t = timeseqs[:elems_per_fold] #time
fold1_t2 = timeseqs2[:elems_per_fold]
fold1_t3 = timeseqs3[:elems_per_fold]
fold1_t4 = timeseqs4[:elems_per_fold]

with open('output_files/folds/fold1.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in izip(fold1, fold1_t):    
        spamwriter.writerow([unicode(s).encode("utf-8") +'#{}'.format(t) for s, t in izip(row, timeseq)])

In [28]:
fold2 = lines[elems_per_fold:2*elems_per_fold]
fold2_t = timeseqs[elems_per_fold:2*elems_per_fold]
fold2_t2 = timeseqs2[elems_per_fold:2*elems_per_fold]
fold2_t3 = timeseqs3[elems_per_fold:2*elems_per_fold]
fold2_t4 = timeseqs4[elems_per_fold:2*elems_per_fold]

with open('output_files/folds/fold2.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in izip(fold2, fold2_t):
        spamwriter.writerow([unicode(s).encode("utf-8") +'#{}'.format(t) for s, t in izip(row, timeseq)])

In [29]:
fold3 = lines[2*elems_per_fold:]
fold3_t = timeseqs[2*elems_per_fold:]
fold3_t2 = timeseqs2[2*elems_per_fold:]
fold3_t3 = timeseqs3[2*elems_per_fold:]
fold3_t4 = timeseqs4[2*elems_per_fold:]

with open('output_files/folds/fold3.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for row, timeseq in izip(fold3, fold3_t):
        spamwriter.writerow([unicode(s).encode("utf-8") +'#{}'.format(t) for s, t in izip(row, timeseq)])

In [30]:
lines = fold1 + fold2 #activity
lines_t = fold1_t + fold2_t #time
lines_t2 = fold1_t2 + fold2_t2
lines_t3 = fold1_t3 + fold2_t3
lines_t4 = fold1_t4 + fold2_t4 #weekday

In [31]:
print('lines: {}'.format(lines))
print('lines_t: {}'.format(lines_t))
print('lines_t2: {}'.format(lines_t2))
print('lines_t3: {}'.format(lines_t3))
print('lines_t4: {}'.format(lines_t4))

lines: [u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa4\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7']
lines_t: [[0, 15, 173999], [0, 457325, 306], [0, 62954, 432], [0, 2221, 109, 1660, 6746], [0, 2697, 64551], [0, 3313, 1141], [0, 13, 1459434, 104956], [0, 3196606, 7], [0, 53, 66609], [0, 14, 315]]
lines_t2: [[0, 15, 174014], [0, 457325, 457631], [0, 62954, 63386], [0, 2221, 2330, 3990, 10736], [0, 2697, 67248], [0, 3313, 4454], [0, 13, 1459447, 1564403], [0, 3196606, 3196613], [0, 53, 66662], [0, 14, 329]]
lines_t3: [[60938, 60953, 62152], [65646, 4571, 4877], [84713, 61267, 61699], [76112, 78333, 78442, 80102, 448], [83638, 86335, 64486], [59432, 62745, 63886], [75741, 75754, 66388, 84944], [64220, 64026, 64033], [1481, 1534, 68143], [78428, 78442, 78757]]
lines_t4: [[1, 1, 3], [4, 3, 3], [2, 3, 3], [1, 1, 1, 1, 2], [1, 1, 2], [3, 3, 3], [4, 4, 0, 1], [2, 4, 4], [3, 3, 3], [1, 1, 1]]


In [32]:
step = 1
sentences = []
softness = 0
next_chars = []
sentences_t = []
sentences_t2 = []
sentences_t3 = []
sentences_t4 = []
next_chars_t = []
next_chars_t2 = []
next_chars_t3 = []
next_chars_t4 = []

In [33]:
lines = map(lambda x: x+'!',lines)

In [34]:
for line, line_t, line_t2, line_t3, line_t4 in izip(lines, lines_t, lines_t2, lines_t3, lines_t4):
    for i in range(0, len(line), step):
        if i==0:
            continue
        sentences.append(line[0: i])
        sentences_t.append(line_t[0:i])
        sentences_t2.append(line_t2[0:i])
        sentences_t3.append(line_t3[0:i])
        sentences_t4.append(line_t4[0:i])
        next_chars.append(line[i])
        
        print('--------------Check----------------')
        print('sentences: {}'.format(sentences))
        print('sentences_t: {}'.format(sentences_t))
        print('sentences_t2: {}'.format(sentences_t2))
        print('sentences_t3: {}'.format(sentences_t3))
        print('sentences_t4: {}'.format(sentences_t4))
        
        if i==len(line)-1: # special case to deal time of end character
            next_chars_t.append(0)
            next_chars_t2.append(0)
            next_chars_t3.append(0)
            next_chars_t4.append(0)
        else:
            next_chars_t.append(line_t[i])
            next_chars_t2.append(line_t2[i])
            next_chars_t3.append(line_t3[i])
            next_chars_t4.append(line_t4[i])
        
        print('--------------Check----------------')
        print('next_chars_t: {}'.format(next_chars_t))
        print('next_chars_t2: {}'.format(next_chars_t2))
        print('next_chars_t3: {}'.format(next_chars_t3))
        print('next_chars_t4: {}'.format(next_chars_t4))
        
print('nb sequences:', len(sentences))

--------------Check----------------
sentences: [u'\xa2']
sentences_t: [[0]]
sentences_t2: [[0]]
sentences_t3: [[60938]]
sentences_t4: [[1]]
--------------Check----------------
next_chars_t: [15]
next_chars_t2: [15]
next_chars_t3: [60953]
next_chars_t4: [1]
--------------Check----------------
sentences: [u'\xa2', u'\xa2\xa9']
sentences_t: [[0], [0, 15]]
sentences_t2: [[0], [0, 15]]
sentences_t3: [[60938], [60938, 60953]]
sentences_t4: [[1], [1, 1]]
--------------Check----------------
next_chars_t: [15, 173999]
next_chars_t2: [15, 174014]
next_chars_t3: [60953, 62152]
next_chars_t4: [1, 3]
--------------Check----------------
sentences: [u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7']
sentences_t: [[0], [0, 15], [0, 15, 173999]]
sentences_t2: [[0], [0, 15], [0, 15, 174014]]
sentences_t3: [[60938], [60938, 60953], [60938, 60953, 62152]]
sentences_t4: [[1], [1, 1], [1, 1, 3]]
--------------Check----------------
next_chars_t: [15, 173999, 0]
next_chars_t2: [15, 174014, 0]
next_chars_t3: [60953, 62152

**Input and output before vectorization are below**

In [51]:
print(sentences)

[u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7\xa9', u'\xa2\xa9\xa7\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa4', u'\xa4\xa2', u'\xa4\xa2\xa9', u'\xa4\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7']


In [52]:
print(sentences_t)

[[0], [0, 15], [0, 15, 173999], [0], [0, 457325], [0, 457325, 306], [0], [0, 62954], [0, 62954, 432], [0], [0, 2221], [0, 2221, 109], [0, 2221, 109, 1660], [0, 2221, 109, 1660, 6746], [0], [0, 2697], [0, 2697, 64551], [0], [0, 3313], [0, 3313, 1141], [0], [0, 13], [0, 13, 1459434], [0, 13, 1459434, 104956], [0], [0, 3196606], [0, 3196606, 7], [0], [0, 53], [0, 53, 66609], [0], [0, 14], [0, 14, 315]]


In [53]:
print(sentences_t2)

[[0], [0, 15], [0, 15, 174014], [0], [0, 457325], [0, 457325, 457631], [0], [0, 62954], [0, 62954, 63386], [0], [0, 2221], [0, 2221, 2330], [0, 2221, 2330, 3990], [0, 2221, 2330, 3990, 10736], [0], [0, 2697], [0, 2697, 67248], [0], [0, 3313], [0, 3313, 4454], [0], [0, 13], [0, 13, 1459447], [0, 13, 1459447, 1564403], [0], [0, 3196606], [0, 3196606, 3196613], [0], [0, 53], [0, 53, 66662], [0], [0, 14], [0, 14, 329]]


In [54]:
print(sentences_t3)

[[60938], [60938, 60953], [60938, 60953, 62152], [65646], [65646, 4571], [65646, 4571, 4877], [84713], [84713, 61267], [84713, 61267, 61699], [76112], [76112, 78333], [76112, 78333, 78442], [76112, 78333, 78442, 80102], [76112, 78333, 78442, 80102, 448], [83638], [83638, 86335], [83638, 86335, 64486], [59432], [59432, 62745], [59432, 62745, 63886], [75741], [75741, 75754], [75741, 75754, 66388], [75741, 75754, 66388, 84944], [64220], [64220, 64026], [64220, 64026, 64033], [1481], [1481, 1534], [1481, 1534, 68143], [78428], [78428, 78442], [78428, 78442, 78757]]


In [55]:
print(sentences_t4)

[[1], [1, 1], [1, 1, 3], [4], [4, 3], [4, 3, 3], [2], [2, 3], [2, 3, 3], [1], [1, 1], [1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 2], [1], [1, 1], [1, 1, 2], [3], [3, 3], [3, 3, 3], [4], [4, 4], [4, 4, 0], [4, 4, 0, 1], [2], [2, 4], [2, 4, 4], [3], [3, 3], [3, 3, 3], [1], [1, 1], [1, 1, 1]]


In [56]:
print(next_chars)

[u'\xa9', u'\xa7', u'!', u'\xa9', u'\xa7', u'!', u'\xa9', u'\xa7', u'!', u'\xa9', u'\xa7', u'\xa9', u'\xa7', u'!', u'\xa9', u'\xa7', u'!', u'\xa9', u'\xa7', u'!', u'\xa2', u'\xa9', u'\xa7', u'!', u'\xa9', u'\xa7', u'!', u'\xa9', u'\xa7', u'!', u'\xa9', u'\xa7', u'!']


In [57]:
print(next_chars_t)

[15, 173999, 0, 457325, 306, 0, 62954, 432, 0, 2221, 109, 1660, 6746, 0, 2697, 64551, 0, 3313, 1141, 0, 13, 1459434, 104956, 0, 3196606, 7, 0, 53, 66609, 0, 14, 315, 0]


In [58]:
print(next_chars_t2)

[15, 174014, 0, 457325, 457631, 0, 62954, 63386, 0, 2221, 2330, 3990, 10736, 0, 2697, 67248, 0, 3313, 4454, 0, 13, 1459447, 1564403, 0, 3196606, 3196613, 0, 53, 66662, 0, 14, 329, 0]


In [59]:
print(next_chars_t3)

[60953, 62152, 0, 4571, 4877, 0, 61267, 61699, 0, 78333, 78442, 80102, 448, 0, 86335, 64486, 0, 62745, 63886, 0, 75754, 66388, 84944, 0, 64026, 64033, 0, 1534, 68143, 0, 78442, 78757, 0]


In [60]:
print(next_chars_t4)

[1, 3, 0, 3, 3, 0, 3, 3, 0, 1, 1, 1, 2, 0, 1, 2, 0, 3, 3, 0, 4, 0, 1, 0, 4, 4, 0, 3, 3, 0, 1, 1, 0]


# Vectorization

chars: list of activities

In [45]:
print('Vectorization...')
num_features = len(chars)+5

Vectorization...


In [46]:
print('num features: {}'.format(num_features))

num features: 9


In [47]:
print('sentences: {}'.format(sentences))

sentences: [u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2\xa9\xa7\xa9', u'\xa2\xa9\xa7\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa4', u'\xa4\xa2', u'\xa4\xa2\xa9', u'\xa4\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7', u'\xa2', u'\xa2\xa9', u'\xa2\xa9\xa7']


In [48]:
X = np.zeros((len(sentences), maxlen, num_features), dtype=np.float32)
y_a = np.zeros((len(sentences), len(target_chars)), dtype=np.float32)
y_t = np.zeros((len(sentences)), dtype=np.float32)

for i, sentence in enumerate(sentences):
    #print('i: {0}, sentence: {1}'.format(i, sentence))
    leftpad = maxlen-len(sentence)
    next_t = next_chars_t[i]
    sentence_t = sentences_t[i]
    sentence_t2 = sentences_t2[i]
    sentence_t3 = sentences_t3[i]
    sentence_t4 = sentences_t4[i]
    
    print('--------------Check----------------')
    print('leftpad: {}'.format(leftpad))
    print('next_t: {}'.format(next_t))
    print('sentence_t: {}'.format(sentence_t))
    print('sentence_t2: {}'.format(sentence_t2))
    print('sentence_t3: {}'.format(sentence_t3))
    print('sentence_t4: {}'.format(sentence_t4))
    
    for t, char in enumerate(sentence):
        multiset_abstraction = Counter(sentence[:t+1])
        
        for c in chars:
            if c==char:
                X[i, t+leftpad, char_indices[c]] = 1
                
        X[i, t+leftpad, len(chars)] = t+1
        X[i, t+leftpad, len(chars)+1] = sentence_t[t]/divisor
        X[i, t+leftpad, len(chars)+2] = sentence_t2[t]/divisor2
        X[i, t+leftpad, len(chars)+3] = sentence_t3[t]/86400
        X[i, t+leftpad, len(chars)+4] = sentence_t4[t]/7
        
    for c in target_chars:
        if c==next_chars[i]:
            y_a[i, target_char_indices[c]] = 1-softness
        else:
            y_a[i, target_char_indices[c]] = softness/(len(target_chars)-1)
    y_t[i] = next_t/divisor
    np.set_printoptions(threshold=np.nan)

--------------Check----------------
leftpad: 5
next_t: 15
sentence_t: [0]
sentence_t2: [0]
sentence_t3: [60938]
sentence_t4: [1]
--------------Check----------------
leftpad: 4
next_t: 173999
sentence_t: [0, 15]
sentence_t2: [0, 15]
sentence_t3: [60938, 60953]
sentence_t4: [1, 1]
--------------Check----------------
leftpad: 3
next_t: 0
sentence_t: [0, 15, 173999]
sentence_t2: [0, 15, 174014]
sentence_t3: [60938, 60953, 62152]
sentence_t4: [1, 1, 3]
--------------Check----------------
leftpad: 5
next_t: 457325
sentence_t: [0]
sentence_t2: [0]
sentence_t3: [65646]
sentence_t4: [4]
--------------Check----------------
leftpad: 4
next_t: 306
sentence_t: [0, 457325]
sentence_t2: [0, 457325]
sentence_t3: [65646, 4571]
sentence_t4: [4, 3]
--------------Check----------------
leftpad: 3
next_t: 0
sentence_t: [0, 457325, 306]
sentence_t2: [0, 457325, 457631]
sentence_t3: [65646, 4571, 4877]
sentence_t4: [4, 3, 3]
--------------Check----------------
leftpad: 5
next_t: 62954
sentence_t: [0]
sentence

In [49]:
target_chars

[u'!', u'\xa2', u'\xa4', u'\xa7', u'\xa9']

In [50]:
print(X)

[[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00]
  [  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     0.00000000e+00]
  [  1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
     1.00000000e+00   0.00000000e+00   0.00000000e+00   7.05300927e-01
     1.42857149e-01]]

 [[  0.00000000e+00 

# Build model

In [ ]:
# build the model: 
print('Build model...')
main_input = Input(shape=(maxlen, num_features), name='main_input')
# train a 2-layer LSTM with one shared layer
l1 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=True, dropout_W=0.2)(main_input) # the shared layer
b1 = BatchNormalization()(l1)
l2_1 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=False, dropout_W=0.2)(b1) # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)
l2_2 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=False, dropout_W=0.2)(b1) # the layer specialized in time prediction
b2_2 = BatchNormalization()(l2_2)
act_output = Dense(len(target_chars), activation='softmax', init='glorot_uniform', name='act_output')(b2_1)
time_output = Dense(1, init='glorot_uniform', name='time_output')(b2_2)

model = Model(input=[main_input], output=[act_output, time_output])

opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004, clipvalue=3)

model.compile(loss={'act_output':'categorical_crossentropy', 'time_output':'mae'}, optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=42)
model_checkpoint = ModelCheckpoint('output_files/models/model_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

model.fit(X, {'act_output':y_a, 'time_output':y_t}, validation_split=0.2, verbose=2, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, nb_epoch=500)